In [1]:
from google.colab import userdata
HF_KEY = userdata.get('HF_KEY')

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!huggingface-cli login --token {HF_KEY}

In [ ]:
!pip install ipywidgets bitsandbytes
!pip install accelerate -U
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface

In [ ]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

selected_model = "meta-llama/Llama-2-7b-hf"

SYSTEM_PROMPT = """Você é um assistente de IA que responde a perguntas de maneira amigável, com base nos documentos fornecidos. Aqui estão algumas regras que você sempre segue:
- Gerar saídas legíveis para humanos, evitando criar texto sem sentido.
- Gerar apenas a saída solicitada, sem incluir qualquer outro idioma antes ou depois da saída solicitada.
- Nunca agradecer, expressar felicidade em ajudar, mencionar que é um agente de IA, etc. Apenas responda diretamente.
- Gerar linguagem profissional geralmente usada em documentos comerciais na América do Norte.
- Nunca gerar linguagem ofensiva ou obscena.
- Traduza as suas respostas sempre para Português Brasileiro. Nunca responsa nada em inglês.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT +"<</SYS>>\n\n{query_str}[/INST] "
)


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    # model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="neuralmind/bert-base-portuguese-cased")

In [15]:
from llama_index.core import VectorStoreIndex
import os
import shutil
import numpy as np
from uuid import uuid4
from llama_index.core import SimpleDirectoryReader

pdf = SimpleDirectoryReader('./data').load_data()

index = VectorStoreIndex.from_documents(pdf)

In [16]:
query_engine = index.as_query_engine()

In [17]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from IPython.display import Markdown, display
response = query_engine.query("Qual o assunto do PDF?")
display(Markdown(f"<b>{response}</b>"))